In [1]:
# add path (for local)
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [2]:
guacamol_oracle_names = ["albuterol_similarity", "amlodipine_mpo", "celecoxib_rediscovery", "deco_hop", "fexofenadine_mpo", "isomers_c7h8n2o2", "isomers_c9h10n2o2pf2cl", "median1", "median2", "mestranol_similarity", "osimertinib_mpo", "perindopril_mpo", "ranolazine_mpo", "scaffold_hop", "sitagliptin_mpo", "thiothixene_rediscovery", "troglitazone_rediscovery", "valsartan_smarts", "zaleplon_mpo"]
tdc_oracle_names = ["drd2", "gsk3b", "jnk3", "qed"]
oracle_names = guacamol_oracle_names + tdc_oracle_names
    
def reward_class_name_from_oracle_name(oracle_name: str) -> str:
    if oracle_name in tdc_oracle_names:
        return "TDCReward"
    else:
        return "GuacaMolReward"

In [ ]:
import os
import optuna
from rdkit import RDLogger
from utils import generator_from_conf, conf_from_yaml
RDLogger.DisableLog('rdApp.*')

yaml_path = "config/mol_opt/optuna_rnn_only.yaml"
oracle_name = None

def objective(trial):
    conf = conf_from_yaml(yaml_path, repo_root)
    conf.setdefault("transition_args", {})
    conf["transition_args"]["sharpness"] = trial.suggest_float("sharpness", 0.8, 1.1)
    conf["transition_args"]["top_p"] = trial.suggest_float("top_p", 0.994, 0.999)
    
    conf.setdefault("policy_args", {})
    conf["policy_args"]["c"] = trial.suggest_float("c", 0.05, 0.4)
    conf["policy_args"]["best_rate"] = trial.suggest_float("best_rate", 0, 1)
    conf["policy_args"]["prior"] = trial.suggest_float("prior", 0.3, 1.2)
    conf["policy_args"]["prior_weight"] = trial.suggest_int("prior_weight", 0, 2)
    conf["policy_args"]["max_prior"] = trial.suggest_float("max_prior", 0, 0.6)

    conf.setdefault("generator_args", {})
    conf["generator_args"]["eval_width"] = trial.suggest_int("eval_width", 1, 40)
    conf["generator_args"]["n_evals"] = trial.suggest_int("n_evals", 1, 10)
    conf["generator_args"]["n_tries"] = trial.suggest_int("n_tries", 1, 10)
    conf["generator_args"]["terminal_reward"] = trial.suggest_categorical("terminal_reward", ["ignore", -1])
    
    conf["reward_class"] = reward_class_name_from_oracle_name(oracle_name)
    conf["reward_args"] = {}
    conf["reward_args"]["objective"] = oracle_name
    conf["output_dir"] = conf.get("output_dir") + os.sep + oracle_name
        
    generator = generator_from_conf(conf, repo_root)
    generator.logger.info("reward="+oracle_name)
    generator.logger.info(f"params={trial.params}")
    max_generations, time_limit = 10000, conf.get("time_limit")
    
    generator.generate(max_generations=1000, time_limit=time_limit/10)
    intermediate_value = generator.top_k_auc(top_k=10, max_oracle_calls=1000)
    trial.report(intermediate_value, 0)
    if trial.should_prune():
        print(f"{oracle_name} Trial {trial.number} - Step {0}: intermediate_score={intermediate_value:.3f}, params={trial.params}")
        raise optuna.TrialPruned()
    
    generator.generate(max_generations=2000, time_limit=time_limit*2/10)
    intermediate_value = generator.top_k_auc(top_k=10, max_oracle_calls=3000)
    trial.report(intermediate_value, 1)
    if trial.should_prune():
        print(f"{oracle_name} Trial {trial.number} - Step {1}: intermediate_score={intermediate_value:.3f}, params={trial.params}")
        raise optuna.TrialPruned()
    
    generator.generate(max_generations=7000, time_limit=time_limit*7/10)
    score = generator.top_k_auc(top_k=10, max_oracle_calls=10000)
    generator.logger.info("top_10_auc: " + str(score))

    trial.set_user_attr("score", score)
    print(f"{oracle_name} Trial {trial.number}: score={score:.3f}")
    
    return score

In [ ]:
# start
for oracle in oracle_names:
    oracle_name = oracle
    name = oracle + "_rnn_only"
    storage = "sqlite:///optuna/mol_opt/" + name + ".db"
    sampler = sampler=optuna.samplers.TPESampler(multivariate=True, group=True)
    pruner = optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=0, interval_steps=1)
    study = optuna.create_study(direction="maximize", study_name=name, storage=storage, sampler=sampler, pruner=pruner)
    study.enqueue_trial({"sharpness": 1.0, "top_p": 0.995, "c": 0.2, "best_rate": 0.5, "prior": 1.0, "prior_weight": 0, "max_prior": 0.5, "eval_width": 1, "n_evals": 1, "n_tries": 5})
    study.optimize(objective, n_trials=1)
    
# continue
for i in range(1, 10000):
    for oracle in oracle_names:
        oracle_name = oracle
        name = oracle + "_rnn_only"
        study = optuna.study.load_study(study_name=name, storage="sqlite:///optuna/mol_opt/" + name + ".db")
        study.optimize(objective, n_trials=i)

/opt/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
/opt/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-07-17 14:39:28,654] A new study created in RDB with name: albuterol_similarity_rnn_only
[I 2025-07-17 14:41:14,823] Trial 0 finished with value: 0.5639513068043246 and parameters: {'sharpness': 1.0, 'top_p': 0.995, 'c': 0.2, 'best_rate': 0.5, 'prior': 1.0, 'prior_weight': 0, 'max_prior': 0.5, 'eval_width': 1, 'n_evals': 1, 'n_tries': 5, 'terminal_reward': -1}. Best is trial 0 with value: 0.5639513068043246.
/opt/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental 

Trial 0: score=0.564


[I 2025-07-17 14:43:07,026] Trial 0 finished with value: 0.4891239588423812 and parameters: {'sharpness': 1.0, 'top_p': 0.995, 'c': 0.2, 'best_rate': 0.5, 'prior': 1.0, 'prior_weight': 0, 'max_prior': 0.5, 'eval_width': 1, 'n_evals': 1, 'n_tries': 5, 'terminal_reward': -1}. Best is trial 0 with value: 0.4891239588423812.
[I 2025-07-17 14:43:07,078] A new study created in RDB with name: celecoxib_rediscovery_rnn_only


Trial 0: score=0.489
